# Combinations API

The Combinations API in Hypster allows you to generate and query all possible combinations of hyperparameters in your configuration. This is particularly useful for hyperparameter tuning and exploring different model configurations.

## Generating Combinations

### Using `get_combinations()`

The `get_combinations()` method generates all possible combinations of hyperparameters for a given configuration.

Example:

```python
from hypster import HP, config, save

@config
def model_config(hp: HP):
    model_type = hp.select(["cnn", "rnn"], default="cnn")
    if model_type == "cnn":
        num_layers = hp.select([3, 5], default=3)
    else:
        cell_type = hp.select(["lstm", "gru"], default="lstm")
    
    learning_rate = hp.number_input(default=0.001)

save(model_config, "model_config.py")

# Generate combinations
combinations = model_config.get_combinations()
print(combinations)
```

Output:
```python
[
    {'model_type': 'cnn', 'num_layers': 3, 'learning_rate': 0.001},
    {'model_type': 'cnn', 'num_layers': 5, 'learning_rate': 0.001},
    {'model_type': 'rnn', 'cell_type': 'lstm', 'learning_rate': 0.001},
    {'model_type': 'rnn', 'cell_type': 'gru', 'learning_rate': 0.001}
]
```

### Nested Configurations

For nested configurations using `hp.propagate()`, the combinations will include the nested structure:

```python
@config
def training_config(hp: HP):
    import hypster
    
    batch_size = hp.select([32, 64], default=32)
    model_config = hypster.load("model_config.py")
    model_params = hp.propagate(model_config, name="model")

# Generate combinations
combinations = training_config.get_combinations()
print(combinations)
```

Output:
```python
[
    {'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 3, 'model.learning_rate': 0.001},
    {'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 5, 'model.learning_rate': 0.001},
    {'batch_size': 32, 'model.model_type': 'rnn', 'model.cell_type': 'lstm', 'model.learning_rate': 0.001},
    {'batch_size': 32, 'model.model_type': 'rnn', 'model.cell_type': 'gru', 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 3, 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 5, 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'rnn', 'model.cell_type': 'lstm', 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'rnn', 'model.cell_type': 'gru', 'model.learning_rate': 0.001}
]
```

## Querying Combinations

The `query_combinations()` function allows you to filter the generated combinations based on specific criteria.

### Using `query_combinations()`

```python
from hypster import query_combinations

filtered_combinations = query_combinations(combinations, {"model.model_type": "cnn"})
print(filtered_combinations)
```

Output:
```python
[
    {'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 3, 'model.learning_rate': 0.001},
    {'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 5, 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 3, 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 5, 'model.learning_rate': 0.001}
]
```

You can use multiple criteria in the query:

```python
filtered_combinations = query_combinations(combinations, {"model.model_type": "cnn", "batch_size": 64})
print(filtered_combinations)
```

Output:
```python
[
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 3, 'model.learning_rate': 0.001},
    {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 5, 'model.learning_rate': 0.001}
]
```

## Best Practices

1. Use `get_combinations()` to explore all possible configurations of your model.
2. Leverage `query_combinations()` to filter and focus on specific subsets of configurations.
3. When dealing with large configuration spaces, consider using these functions in combination with distributed computing frameworks for efficient hyperparameter tuning.
4. Remember that the number of combinations can grow exponentially with the number of hyperparameters. Use conditional hyperparameters (like in the `model_config` example) to keep the combination space manageable.

By using the Combinations API, you can systematically explore and experiment with different configurations of your models, leading to more thorough and efficient hyperparameter tuning processes.